In [ ]:
# 필요한 패키지 설치
%pip install -U anthropic openai google-generativeai langchain-openai langchain-anthropic langchain-google-genai

In [ ]:
from dotenv import load_dotenv

load_dotenv()

### OpenAI API 활용

In [ ]:
# OpenAI API 사용 예제
import openai
from IPython.display import display, Markdown
# API 키 설정
client = openai.OpenAI()

# GPT-4o 모델 호출
response_gpt4o = client.chat.completions.create(
    model="gpt-4o",
    temperature=0.7,
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": "파이썬으로 퀵소트 알고리즘을 구현해주세요."},
    ]
)

display(Markdown(f"### 💬 답변\n\n{response_gpt4o.choices[0].message.content}"))

In [3]:
# o3-mini 모델 호출 (올바른 모델명 사용)
response_o3mini = client.responses.create(
    model="o3-mini",  # o3-mini-high가 아닌 o3-mini 사용
    reasoning={"effort": "low"}, #low, medium, high 사용 가능
    input=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": "파이썬으로 퀵소트 알고리즘을 구현해주세요."}
    ]
)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"### 💬 답변\n\n{response_o3mini.output_text}"))

### Claude API 활용하기

In [ ]:
# Claude API 사용 예제
import anthropic

# API 키 설정
client = anthropic.Anthropic()

from IPython.display import display, Markdown

def render_response(blocks):
    for block in blocks:
        if getattr(block, "type", None) == "thinking":
            display(Markdown(f"### 🤔 Thinking\n\n{block.thinking}"))
        elif getattr(block, "type", None) == "text":
            display(Markdown(f"### 💬 답변\n\n{block.text}"))

# Claude 3.7 Sonnet 호출 예시
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",  # Claude 3.7 Sonnet 모델
    max_tokens=1000,
    temperature=0.7,
    system="당신은 유용한 AI 비서입니다.",
    messages=[
        {"role": "user", "content": "파이썬으로 피보나치 수열을 계산하는 코드를 작성해주세요."}
    ]
)

# 응답 출력
render_response(message.content)

In [ ]:
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=3000,
    thinking={
        "type": "enabled",
        "budget_tokens": 2000
    },
    messages=[
        {"role": "user", "content": "파이썬으로 피보나치 수열을 계산하는 코드를 작성해주세요."}
    ]
)

render_response(response.content)

### Gemini API 활용하기

In [ ]:
# Gemini API 사용 예제
from google import genai

# API 키 설정
client = genai.Client()

# Gemini 모델 호출 (올바른 모델명 사용)
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["파이썬으로 이진 검색 알고리즘을 구현해주세요."],
)

print("Gemini 2.0 Flash 응답:")
display(Markdown(f"### 💬 답변\n\n{response.text}"))

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash-thinking-exp",
    contents=["파이썬으로 이진 검색 알고리즘을 구현해주세요."],
)

print("gemini-2.0-flash")
display(Markdown(f"### 💬 답변\n\n{response.text}"))

In [ ]:
response

### Langchain을 활용한 Closed LLM 활용법

#### 일반 모델 활용하기

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 설정 (환경 변수에서 가져옴)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# 모델 초기화
gpt4o = ChatOpenAI(model="gpt-4o")
claude = ChatAnthropic(model="claude-3-haiku-20240307")
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# 질문 설정
question = "파이썬으로 이진 검색 알고리즘을 구현해주세요."

# 모델 리스트 생성
models = [
    {"name": "GPT-4o", "model": gpt4o},
    {"name": "Claude", "model": claude},
    {"name": "Gemini", "model": gemini}
]

# 각 모델에 질문하고 응답 받기
for model_info in models:
    print(f"\n{model_info['name']} 응답:")
    for token in model_info["model"].stream(question):
        print(token.content, end="")
    print("\n" + "-"*50)

#### 추론 모델 활용하기

#### OpenAI o3-mini

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
o3_mini = ChatOpenAI(model="o3-mini")

for token in o3_mini.stream(
    "파이썬으로 퀵소트 알고리즘을 구현해주세요.",
    reasoning_effort="medium"
    ):
    print(token.content, end="")

#### Anthropic Claude 3.7 sonnet

In [ ]:
from langchain_anthropic import ChatAnthropic

claude_3_7 = ChatAnthropic(
    model="claude-3-7-sonnet-latest",
    max_tokens=5000,
    thinking={"type": "enabled", "budget_tokens": 2000},
)


for token in claude_3_7.stream(
    "파이썬으로 퀵소트 알고리즘을 구현해주세요.",
    ):
    print(token.content, end="")

#### Google Gemini 2.0 Flash Thinking

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_2_0 = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-thinking-exp",
)

for token in gemini_2_0.stream(
    "파이썬으로 이진 검색 알고리즘을 구현해주세요.",
    ):
    print(token.content, end="")

### Langchain의 init_chat_model로 동일한 형식의 모델 선언하기

#### 일반 모델

In [ ]:
from langchain.chat_models import init_chat_model

# langchain_openai.ChatOpenAI 인스턴스를 반환합니다.
gpt_4o = init_chat_model("gpt-4o", model_provider="openai", temperature=0)
# langchain_anthropic.ChatAnthropic 인스턴스를 반환합니다.
claude_3_7 = init_chat_model(
    "claude-3-7-sonnet-latest", model_provider="anthropic", temperature=0
)
# langchain_google_genai.ChatGoogleGenerativeAI 인스턴스를 반환합니다.
gemini_2_0 = init_chat_model(
    "gemini-2.0-flash-thinking-exp", model_provider="google_genai", temperature=0
)

# 모든 모델 통합은 ChatModel 인터페이스를 구현하므로 동일한 방식으로 사용할 수 있습니다.
print("GPT-4o: " + gpt_4o.invoke("당신의 이름은 무엇인가요?").content + "\n")
print("Claude 3.7: " + claude_3_7.invoke("당신의 이름은 무엇인가요?").content + "\n")
print("Gemini 2.0: " + gemini_2_0.invoke("당신의 이름은 무엇인가요?").content + "\n")

#### 추론 모델

In [21]:
o3_mini = init_chat_model(
    "openai:o3-mini",
    configurable_fields="any",
    config_prefix="model"
)

claude_3_7 = init_chat_model(
    "anthropic:claude-3-7-sonnet-latest",
    configurable_fields="any",
    config_prefix="model"
)

gemini_2_0 = init_chat_model(
    "google_genai:gemini-2.0-flash-thinking-exp",
    configurable_fields="any",
    config_prefix="model"
)

In [ ]:
o3_mini_response = o3_mini.invoke(
    "피보나치 수열을 파이썬으로 구현해주세요.",
    config={
        "configurable": {
            "reasoning_effort": "low"
        }
    }
)

claude_3_7_response = claude_3_7.invoke(
    "피보나치 수열을 파이썬으로 구현해주세요.",
    config={
        "configurable": {
            "max_tokens":5000,
            "thinking":{"type": "enabled", "budget_tokens": 2000},
        }
    }
)

print(f"o3-mini: {o3_mini_response.content}")
print("-"*50)
print(f"claude-3-7-sonnet-latest: {claude_3_7_response.content}")
print("-"*50)